# webserver-1      | exec /opt/airflow/script/entrypoint.sh: no such file or directory
# webserver-1 exited with code 1


Tôi sẽ giải thích chi tiết về lỗi và cách khắc phục:

### 1. Lỗi gặp phải:
```bash
webserver-1      | exec /opt/airflow/script/entrypoint.sh: no such file or directory
webserver-1 exited with code 1
```

### 2. Nguyên nhân:
1. **Sai đường dẫn mount volume:**
```yaml
volumes:
  - ./script/entrypoint.sh:/opt/airflow/script/entrypoint.sh  # Sai
```
- Trong file docker-compose, bạn đang mount file từ `./script/entrypoint.sh` (local) vào `/opt/airflow/script/entrypoint.sh` (container)
- Nhưng folder `script` không tồn tại (nên là `scripts`)
- Container không thể tìm thấy file entrypoint.sh tại đường dẫn đã chỉ định

2. **Cấu trúc thư mục không đúng:**
- Apache Airflow mặc định sử dụng thư mục `scripts` (số nhiều)
- Bạn đang sử dụng `script` (số ít)

### 3. Cách khắc phục:

1. **Sửa cấu trúc thư mục:**
```plaintext
your-project/
├── docker-compose.yml
├── scripts/              # Đổi từ script -> scripts
│   └── entrypoint.sh
├── dags/
└── requirements.txt
```

2. **Sửa docker-compose.yml:**
```yaml:docker-compose.yml
webserver:
    image: apache/airflow:2.6.0-python3.9
    command: webserver
    entrypoint: /opt/airflow/scripts/entrypoint.sh  # Sửa đường dẫn
    volumes:
      - ./dags:/opt/airflow/dags
      - ./scripts/entrypoint.sh:/opt/airflow/scripts/entrypoint.sh  # Sửa đường dẫn
      - ./requirements.txt:/opt/airflow/requirements.txt
    # ... các cấu hình khác ...
```

3. **Hoặc sử dụng entrypoint mặc định:**
```yaml:docker-compose.yml
webserver:
    image: apache/airflow:2.6.0-python3.9
    command: webserver
    # Bỏ dòng entrypoint và volume mount của entrypoint.sh
    volumes:
      - ./dags:/opt/airflow/dags
      - ./requirements.txt:/opt/airflow/requirements.txt
    # ... các cấu hình khác ...
```

### 4. Giải thích cách khắc phục:

1. **Cách 1 - Sửa cấu trúc:**
- Tạo đúng cấu trúc thư mục
- Đảm bảo file entrypoint.sh tồn tại và có quyền thực thi
- Mount volume với đường dẫn chính xác

2. **Cách 2 - Dùng mặc định:**
- Bỏ qua custom entrypoint
- Sử dụng entrypoint mặc định của Airflow
- Đơn giản hóa cấu hình

### 5. Lợi ích sau khi sửa:
- Container sẽ khởi động thành công
- Tránh được lỗi file không tìm thấy
- Cấu trúc project chuẩn hơn
- Dễ maintain và debug hơn

Bạn có thể chọn một trong hai cách trên tùy theo nhu cầu:
- Nếu cần custom entrypoint: dùng cách 1
- Nếu không cần custom: dùng cách 2 (đơn giản hơn)


## sửa thành scripts 

```
broker           |      goal.violation.distribution.threshold.multiplier = 1.1                     
broker           |      goal.violation.exclude.recently.removed.brokers = true                     
broker           |      goals = [com.linkedin.kafka.cruisecontrol.analyzer.goals.MovementExclusionGoal, io.confluent.cruisecontrol.analyzer.goals.ReplicaPlacementGoal, com.linkedin.kafka.cruisecontrol.analyzer.goals.RackAwareGoal, io.confluent.cruisecontrol.analyzer.goals.MaxReplicaMovementParallelismGoal, com.linkedin.kafka.cruisecontrol.analyzer.goals.ReplicaCapacityGoal, com.linkedin.kafka.cruisecontrol.analyzer.goals.DiskCapacityGoal, com.linkedin.kafka.cruisecontrol.analyzer.goals.NetworkInboundCapacityGoal, com.linkedin.kafka.cruisecontrol.analyzer.goals.NetworkOutboundCapacityGoal, com.linkedin.kafka.cruisecontrol.analyzer.goals.ProducerInboundCapacityGoal, com.linkedin.kafka.cruisecontrol.analyzer.goals.SystemTopicEvenDistributionGoal, com.linkedin.kafka.cruisecontrol.analyzer.goals.ReplicaDistributionGoal, com.linkedin.kafka.cruisecontrol.analyzer.goals.DiskUsageDistributionGoal, com.linkedin.kafka.cruisecontrol.analyzer.goals.LeaderReplicaDistributionGoal, com.linkedin.kafka.cruisecontrol.analyzer.goals.NetworkInboundUsageDistributionGoal, com.linkedin.kafka.cruisecontrol.analyzer.goals.NetworkOutboundUsageDistributionGoal, com.linkedin.kafka.cruisecontrol.analyzer.goals.CpuUsageDistributionGoal, com.linkedin.kafka.cruisecontrol.analyzer.goals.TopicReplicaDistributionGoal, com.linkedin.kafka.cruisecontrol.analyzer.goals.LeaderBytesInDistributionGoal]    

dependency failed to start: container bigdata-project-webserver-1 exited (1)
PS D:\OneDrive - Hanoi University of Science and Technology\GIT\BigData-Project> 
```

- Bug gì ??
- nguyên nhân do đâu?
- Cách khắc phục ? tại sao lại khắc phục như thế, đã là cách tối ưu nhất chưa, có chắc là sửa như thế hoạt động không? 


## ấn đề về line endings - trước từng gặp

Tôi sẽ giải thích vấn đề và cách khắc phục bằng tiếng Việt:

### Bug
Lỗi chính: `dependency failed to start: container bigdata-project-webserver-1 exited (1)`
- Container webserver của Airflow không thể khởi động và thoát với mã lỗi 1

### Nguyên nhân
1. Vấn đề với quyền thực thi của file entrypoint.sh
2. Có thể file entrypoint.sh đang sử dụng định dạng Windows line endings (CRLF) thay vì Unix (LF)
3. Đường dẫn trong volume mapping có thể không chính xác


### Phân tích vấn đề và giải pháp:

---

### **Bug**
- **Lỗi chính:** 
  Container `webserver` của Airflow không thể khởi động và thoát với mã lỗi `1`.

- **Thông báo lỗi cụ thể:**  
  `dependency failed to start: container bigdata-project-webserver-1 exited (1)`.

---

### **Nguyên nhân tiềm ẩn**
1. **Quyền thực thi của tệp `entrypoint.sh`:**
   - Tệp `entrypoint.sh` không có quyền thực thi cần thiết, dẫn đến lỗi khi container cố gắng chạy tệp này.

2. **Định dạng dòng tệp `entrypoint.sh`:**
   - Tệp sử dụng định dạng dòng **Windows (CRLF)** thay vì **Unix (LF)**. Điều này làm hỏng tệp script khi chạy trong môi trường Linux-based container.

3. **Volume Mapping không chính xác:**
   - Các đường dẫn trong khai báo volume mapping trong `docker-compose.yml` có thể không chính xác hoặc tệp không tồn tại trong host.

4. **Thiếu quyền ghi trên volume:**
   - Volume được mount nhưng không có quyền đọc/ghi (**read-write: `:rw`**) hoặc bị hạn chế quyền.

---

### **Cách khắc phục**

#### **1. Kiểm tra quyền thực thi của tệp `entrypoint.sh`:**
   - Chạy lệnh dưới đây trên hệ thống máy chủ để đảm bảo tệp có quyền thực thi:
     ```bash
     chmod +x ./scripts/entrypoint.sh
     ```

#### **2. Chuyển định dạng dòng từ CRLF sang LF:**
   - Sử dụng một trong các cách sau:
     - Nếu đang sử dụng **VS Code**:
       1. Mở `entrypoint.sh` trong VS Code.
       2. Ở góc dưới phải, chọn `CRLF` và đổi thành `LF`.
       3. Lưu lại tệp.
     - Sử dụng lệnh **dos2unix**:
       ```bash
       dos2unix ./scripts/entrypoint.sh
       ```

#### **3. Đảm bảo volume mapping chính xác trong `docker-compose.yml`:**
   - Đảm bảo các volume trong `docker-compose.yml` được khai báo đúng và thư mục/tệp trong host có tồn tại:
     ```yaml
     volumes:
       - ./dags:/opt/airflow/dags:rw
       - ./scripts/entrypoint.sh:/opt/airflow/scripts/entrypoint.sh:rw
       - ./requirements.txt:/opt/airflow/requirements.txt:rw
     ```
   - Kiểm tra xem các tệp `dags`, `entrypoint.sh`, và `requirements.txt` có tồn tại trong host hay không:
     ```bash
     ls ./dags ./scripts/entrypoint.sh ./requirements.txt
     ```

#### **4. Đảm bảo quyền đọc/ghi (RW) trên các volume:**
   - Volume được khai báo với quyền `:rw` trong `docker-compose.yml`, ví dụ:
     ```yaml
     volumes:
       - ./scripts/entrypoint.sh:/opt/airflow/scripts/entrypoint.sh:rw
     ```

#### **5. Khởi động lại container sau khi sửa lỗi:**
   - Dừng toàn bộ các container cũ:
     ```bash
     docker-compose down
     ```
   - Xóa container cũ (nếu cần thiết):
     ```bash
     docker rm -f $(docker ps -aq)
     ```
   - Khởi động lại hệ thống:
     ```bash
     docker-compose up --build
     ```

---

### **Kiểm tra tệp `entrypoint.sh`**

#### Nội dung tệp sau khi chỉnh sửa:
```bash
#!/bin/bash
set -e

# Cài đặt pip dependencies nếu có tệp requirements.txt
if [ -e "/opt/airflow/requirements.txt" ]; then
  python -m pip install --upgrade pip
  pip install --user -r /opt/airflow/requirements.txt
fi

# Khởi tạo cơ sở dữ liệu Airflow nếu chưa tồn tại
if [ ! -f "/opt/airflow/airflow.db" ]; then
  airflow db init
  airflow users create \
    --username admin \
    --firstname admin \
    --lastname admin \
    --role Admin \
    --email admin@example.com \
    --password admin
fi

# Cập nhật cơ sở dữ liệu Airflow
airflow db upgrade

# Khởi động webserver
exec airflow webserver
```

#### Lưu ý:
- Đường dẫn `requirements.txt` được sửa thành `/opt/airflow/requirements.txt`.
- Lệnh `exec airflow webserver` đảm bảo rằng webserver trở thành quá trình chính (PID 1) của container, không bị ghi đè.

---

### **Lợi ích của việc thêm `:rw` trong volume**
- **Quyền đọc/ghi (read-write):**  
  Cho phép container sửa đổi nội dung của volume, ví dụ:
  - Thêm file log hoặc dữ liệu vào `dags`.
  - Ghi dữ liệu cập nhật vào `requirements.txt` hoặc các file script.

---

### **Tóm tắt**
| **Nguyên nhân**               | **Cách khắc phục**                                                                                 |
|--------------------------------|---------------------------------------------------------------------------------------------------|
| Quyền thực thi của `entrypoint.sh` | Thêm quyền thực thi: `chmod +x ./scripts/entrypoint.sh`.                                        |
| Định dạng dòng sai             | Chuyển sang Unix (LF) bằng VS Code hoặc lệnh `dos2unix ./scripts/entrypoint.sh`.                  |
| Volume mapping sai             | Kiểm tra và sửa volume mapping trong `docker-compose.yml`, đảm bảo các file/tệp tồn tại trên host.|
| Thiếu quyền RW trên volume     | Thêm `:rw` vào volume trong `docker-compose.yml` để đảm bảo quyền đọc/ghi.                       |

Sau khi thực hiện các bước trên, container `webserver` của Airflow sẽ hoạt động bình thường. Nếu cần hỗ trợ thêm, hãy cung cấp log chi tiết từ container bị lỗi.

**Đã fix bằng cách đưa lỗi bắt được từ CURSOR sang GPT4o (context base on o1-mini)**: 
1. Vào file entrypoint.sh trên VSCode sửa CTRF sang LF (pass)
2. Trước đó có thêm quyền ghi 
```
      - ./dags:/opt/airflow/dags:rw
      - ./scripts/entrypoint.sh:/opt/airflow/scripts/entrypoint.sh:rw
      - ./requirements.txt:/opt/airflow/requirements.txt:rw
```

Chắc mình đoán nó ko gây ra impact.


## 3. Bug mới 

```  
control-center   |      rocksdb.config.setter = null
broker           | [2024-11-29 17:20:30,091] INFO [Broker id=1] Handling LeaderAndIsr request correlationId 13 from controller 1 for 1 partitions (state.change.logger)                               
control-center   |      security.protocol = PLAINTEXT
broker           | [2024-11-29 17:20:30,091] TRACE [Broker id=1] Received LeaderAndIsr request LeaderAndIsrPartitionState(topicName='_confluent-controlcenter-7-4-0-1-monitoring-message-rekey-store', topicId=cB57vIcORuuu9J7tWxeX9w, clusterLinkId=null, clusterLinkTopicState=null, linkedLeaderEpoch=-1, clusterLinkSourceTopicId=AAAAAAAAAAAAAAAAAAAAAA, partitionIndex=0, controllerEpoch=1, leader=1, confluentIsUncleanLeader=false, leaderEpoch=0, isr=[1], partitionEpoch=0, replicas=[1], observers=[], addingReplicas=[], removingReplicas=[], isNew=true, leaderRecoveryState=0) correlation id 13 from controller 1 epoch 1 (state.change.logger)
dependency failed to start: container bigdata-project-webserver-1 is unhealthy
PS D:\OneDrive - Hanoi University of Science and Technology\GIT\BigData-Project> 
```

Check docker desktop thấy port: 
```
schema-registry
774b4c75a0ab
confluentinc/cp-schema-registry:7.4.0
8081:8081⁠
```

bị tắt => bật chay cái là được